In [1]:
# imports
import keras
import numpy as np
import pandas as pd
import torch_cnn as tcnn
import keras_cnn as kcnn
import data_processing as d
from tensorflow import keras
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split   

In [2]:
# Data retieval
X, y = d.get_encoded_data()

# Keras CNN Data
X_train, X_test, y_train, y_test = d.splitData(X, y)

# Convert data to Tensors to Incease Computation time for Torch CNN
X_tensor_train,Y_tensor_train  = d.toTensors(X, y)

# Change Y data for SVM
svm_X_train, svm_X_test, svm_y_train, svm_y_test =  train_test_split(X, d.get_SVM_y(), test_size=0.1, random_state=4)

In [3]:
# Variables

EPOCHS = 100 # arbitrary

# Keras CNN
KERNEL_SIZE = 3
FILTERS = 128
DROPOUT_RATE = 0.25

# Torch CNN
N_FEATURES = X_train.shape[1] # Number of features for the input layer
NUM_ROWS_TRAINING = X_train.shape[0] # Number of rows
N_HIDDEN = N_FEATURES * 10 # Size of first linear layer
N_CNN_KERNEL, MAX_POOL_KERNEL = 3, 4 # CNN kernel size

In [4]:
# Keras CNN
#Build Model
self1 = kcnn.keras_CNN(X_train, X_test, y_train, y_test, KERNEL_SIZE,10)
results = self1.main()
d.plot(results, 10, "Accuracy")

Epoch 1/10
10/10 [==============================] - 1s 46ms/step - loss: 0.0861 - accuracy: 0.8792
Epoch 2/10
10/10 [==============================] - 0s 39ms/step - loss: 0.0861 - accuracy: 0.8792
Evaluate on test data


ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/engine/training.py", line 1751, in predict_step
        return self(x, training=False)
    File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) fro

In [ ]:
### Torch CNN
# Build CNN
net = tcnn.CNN(n_feature=N_FEATURES, n_hidden=N_HIDDEN, n_output=13, n_cnn_kernel=N_CNN_KERNEL)   # define the network    
# Train and Test
train_results, results = tcnn.trainTestCNN(net,X_tensor_train,Y_tensor_train, EPOCHS)
# Plotting
d.plot(results, 10, "Accuracy")

In [ ]:
# SVM
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto', kernel='rbf'))
accuracy = []
# Train and Test
for x in range(0,EPOCHS):
    clf.fit(svm_X_train,  svm_y_train)
    if x % 10 == 0:
        predictions=clf.predict(svm_X_test)
        accuracy.append(accuracy_score(svm_y_test, predictions))
# Plotting        
results = {'Accuracy':accuracy}
d.plot(results, 10, 'Accuracy')        